##### Importando as bibliotecas necessárias

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from parsel import Selector
import pandas as pd
from time import sleep

##### Abrir página e mudar para o iframe

In [6]:
# Função que inicia o driver:
def iniciar_driver():
    chrome_options = Options()
    # idioma português Brasil e abrir maximiado a tela
    arguments = ['--lang=pt-BR', '--start-maximized']
    for argument in arguments:
        chrome_options.add_argument(argument)
    driver = webdriver.Chrome(options=chrome_options)
    return driver

In [7]:
# inicar driver
site = "https://johnssimplesmentegostoso.com.br/"
driver = iniciar_driver()

In [8]:
# entrar no site
driver.get(site)

In [9]:
# Mudar para o iframe
sleep(3)
driver.switch_to.frame("main")

##### Fazer o Scraping

In [10]:
# Dicionário para armazenar os items
items = {
    "menu_items" : []
}

In [11]:
# Análise do código fonte da página
resposta = Selector(driver.page_source)

**Destaques**

In [13]:
destaques = resposta.xpath("//div[@class='highlight-items-carousel']")
titulo = destaques.xpath("h2/text()").get()
nomes = destaques.xpath("//h4[@class='name']/text()").extract()
precos = destaques.css("div.prices .current::text").extract()
precos = list(map(lambda p: p.replace("R$\xa0", "").replace(",", "."), precos))

In [14]:
# preenchendo o JSON:
for nome, preco in zip(nomes, precos):
    items["menu_items"].append(
        {
            "name" : nome,
            "category" : titulo,
            "price" : preco
        }
    )

In [15]:
items

{'menu_items': [{'name': 'Pudim', 'category': 'Destaques', 'price': '12.00'},
  {'name': 'IRRESISTIVEL', 'category': 'Destaques', 'price': '16.20'}]}

**Outros Itens**

In [16]:
categorias = resposta.css("div.category-container")

In [17]:
for categoria in categorias:
    titulo = categoria.css('h2.title::text').extract_first()
    nomes = categoria.css("div.text-info h4::text").extract()
    precos = categoria.css("div.price p::text").extract()
    precos = list(map(lambda p: p.replace("R$\xa0", "").replace(",", "."), precos))
    dados = [{"name" : nome, "category" : titulo, "price" : preco} for nome, preco in zip(nomes, precos )]
    items["menu_items"] += dados

In [18]:
items

{'menu_items': [{'name': 'Pudim', 'category': 'Destaques', 'price': '12.00'},
  {'name': 'IRRESISTIVEL', 'category': 'Destaques', 'price': '16.20'},
  {'name': 'GORGON ONION CRISPY',
   'category': '>>> Burger do mês <<<',
   'price': '42.00'},
  {'name': 'IRRESISTIVEL',
   'category': 'BARATI$$IMO SMASH DAY',
   'price': ' 18.00 '},
  {'name': 'OPS ! BACON',
   'category': 'BARATI$$IMO SMASH DAY',
   'price': '16.20'},
  {'name': 'MELT BACON',
   'category': 'BARATI$$IMO SMASH DAY',
   'price': '20.00'},
  {'name': 'SMASH BACON',
   'category': 'BARATI$$IMO SMASH DAY',
   'price': '22.00'},
  {'name': 'BATATA D´CUPIM', 'category': 'PRA COMEÇAR', 'price': '34.00'},
  {'name': 'HOT WINGS', 'category': 'PRA COMEÇAR', 'price': '28.00'},
  {'name': 'BATATA CHEDDAR E CRISPY DE BACON',
   'category': 'PRA COMEÇAR',
   'price': '28.00'},
  {'name': 'BOLINHA DE CARNE DO SOL',
   'category': 'PRA COMEÇAR',
   'price': '26.00'},
  {'name': 'ONION RINGS', 'category': 'PRA COMEÇAR', 'price': '20.0

##### Salvando o resultado em JSON

In [19]:
import json

In [22]:
with open("menu.json", "w", encoding='UTF-8') as menu:
    json.dump(items, menu, indent=4, ensure_ascii=False)